In [24]:
import pandas as pd
org_data_path = "../CHESS/data/df_2_just_moves.csv"
df_2 = pd.read_csv(org_data_path)
df_2 = df_2.drop(columns=['moves_fen'])
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   game_id            20058 non-null  int64 
 1   moves              20058 non-null  object
 2   opening_code       20058 non-null  object
 3   opening_moves      20058 non-null  int64 
 4   opening_fullname   20058 non-null  object
 5   opening_shortname  20058 non-null  object
 6   opening_variation  20058 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.1+ MB


In [25]:
from extract_features import get_fen_from_moves

def create_fen_columns(df_moves):
    '''
    Se añaden columnas para cada movimiento de apertura (de 1 a 28 máximo) en formato fen al df_moves proporcionado
    '''
    max_opening_moves = 28//2  # Número máximo de turnos de apertura (1 turno = 2 movimientos)

    # Crear las columnas para cada movimiento de apertura
    for turn_num in range(1, max_opening_moves + 1):
        # Rellenar con None donde no hay movimientos
        df_moves[f'0w_{turn_num}'] = None
        df_moves[f'0w_{turn_num}_fen'] = None
        df_moves[f'1b_{turn_num}'] = None
        df_moves[f'1b_{turn_num}_fen'] = None

    for index, row in df_moves.iterrows():
        moves = row['moves'].split()
        opening_moves = int(row['opening_moves'])

        if opening_moves % 2 == 0:
            opening_turns = opening_moves//2
        else:
            opening_turns = (opening_moves//2) + 1
        
        for turn_num in range(1, opening_turns + 1):
            san_white = moves[:turn_num*2-1]
            san_black = moves[:turn_num*2]
            fen_white = get_fen_from_moves(' '.join(san_white))
            fen_black = get_fen_from_moves(' '.join(san_black))
            df_moves.at[index, f'0w_{turn_num}'] = san_white[-1]
            df_moves.at[index, f'0w_{turn_num}_fen'] = fen_white

            # No rellenar el ultimo movimiento de negras cuando es el ultimo turno pero movimientos son impares
            if opening_moves % 2 != 0 and turn_num == opening_turns:
                break

            df_moves.at[index, f'1b_{turn_num}'] = san_black[-1]
            df_moves.at[index, f'1b_{turn_num}_fen'] = fen_black

In [27]:
create_fen_columns(df_2)
df_2 = df_2.drop(columns=['moves', 'game_id'])
df_2.columns

Index(['opening_code', 'opening_moves', 'opening_fullname',
       'opening_shortname', 'opening_variation', '0w_1', '0w_1_fen', '1b_1',
       '1b_1_fen', '0w_2', '0w_2_fen', '1b_2', '1b_2_fen', '0w_3', '0w_3_fen',
       '1b_3', '1b_3_fen', '0w_4', '0w_4_fen', '1b_4', '1b_4_fen', '0w_5',
       '0w_5_fen', '1b_5', '1b_5_fen', '0w_6', '0w_6_fen', '1b_6', '1b_6_fen',
       '0w_7', '0w_7_fen', '1b_7', '1b_7_fen', '0w_8', '0w_8_fen', '1b_8',
       '1b_8_fen', '0w_9', '0w_9_fen', '1b_9', '1b_9_fen', '0w_10',
       '0w_10_fen', '1b_10', '1b_10_fen', '0w_11', '0w_11_fen', '1b_11',
       '1b_11_fen', '0w_12', '0w_12_fen', '1b_12', '1b_12_fen', '0w_13',
       '0w_13_fen', '1b_13', '1b_13_fen', '0w_14', '0w_14_fen', '1b_14',
       '1b_14_fen'],
      dtype='object')

In [28]:
# Guardar el CSV con features como df_2.csv
df_2.to_csv("../CHESS/data/df_2_just_moves_fen.csv", index=False) # Data frame con features basadas en ventaja posicional del turno